# Mediana dla obrazu kolorowego

Idea filtracji medianowej jest dość prosta dla obrazów w odcieniach szarości.
Dla obrazów kolorowych trudniej jest określić kryterium wg. którego szeregowane będą wartości, z których wyznaczana będzie mediana.

Jedną z możliwości wykonania filtracji medianowej dla obrazów kolorowych (na podstawie *The Image Processing Handbook*, J. Russ) jest wykorzystanie następującej definicji mediany:
``mediana to ten piksel z otoczenia, którego odległość do innych pikseli z otoczenia jest najmniejsza''.
Jako miarę odległości wykorzystujemy pierwiastek z sumy kwadratów różnic poszczególnych składowych R,G,B.
Zatem odległość między dwoma pikselami wyraża się wzorem:
\begin{equation}
dRGB = \sqrt{(R_1-R_2)^2+(G_1-G_2)^2+(B_1-B_2)^2}
\end{equation}

Warto zwrócić uwagę, że istnieje wiele możliwości zdefiniowania porównywania wielkości wektorowych (jeden piksel to wektor o trzech składowych).
Można zamiast odległości wykorzystać kąt albo połączyć oba parametry.
Ponadto istnieje możliwość dodania do wektora dodatkowych składowych - tak aby lepiej opisać piksel.

Celem zadania jest implementacja opisanego algorytmu.

1. Wczytaj obraz *lenaRGBSzum.png* (dostępny na git).
2. Zdefiniuj rozmiar okna.
3. Wykonaj pętle po pikselach, dla których okno jest zdefiniowane (pomiń brzeg obrazu).
4. Dla każdego piksela pobierz okno o właściwym rozmiarze.
5. Wykonaj pętle po oknie, wewnątrz której obliczona zostanie suma odległości.
    - Obliczanie różnicy: `window - window[rowWin, colWin]`.
    - Obliczanie kwadratów: `np.square`.
    - Obliczanie pierwiastka: `np.sqrt`.
    - Obliczanie sumy metodą `.sum`.
6. Po obliczeniu macierzy odległości wyznacz argument elementu minimalnego.
Wykorzystaj funkcję `np.argmin`.
Argument funkcji zostanie spłaszczony, jeśli ma więcej niż jeden wymiar.
Aby przekonwertować spłaszczony indeks na indeks macierzy wykorzystaj funkcję `np.unravel_index`.
7. Przypisz odpowiedni wektor wartości do piksela obrazu wynikowego.
8. Wyświetl obraz oryginalny i przefiltrowany.
9. Przeprowadź dwa eksperymenty - dla obrazu _lenaRGB_ oraz _lenaRGBszum_.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/'

fileNames = ["lenaRGB.png", "lenaRGBSzum.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [ ]:
lena_rgb_szum = cv2.imread('lenaRGBSzum.png')
lena_rgb_szum = cv2.cvtColor(lena_rgb_szum,cv2.COLOR_BGR2RGB)
lena_rgb = cv2.imread('lenaRGB.png')
lena_rgb = cv2.cvtColor(lena_rgb,cv2.COLOR_BGR2RGB)

def RGB_median(image, size):
    out_image = image.copy()
    for i in range((size-1)//2, image.shape[0] - (size-1)//2):
        for j in range((size-1)//2, image.shape[1] - (size-1)//2):
            
            window = (image[i-(size-1)//2:i+(size-1)//2+1, j-(size-1)//2:j+(size-1)//2+1]).astype(float)
            
            mtrx = np.zeros((size,size))
            
            for rowWin in range(size):
                for colWin in range(size):
                    mtrx[rowWin, colWin] = np.sqrt(np.sum(np.square(window - window[rowWin, colWin])))
                      
            indices = np.unravel_index(np.argmin(mtrx), (size,size))
            
            out_image[i, j] = window[indices[0], indices[1]]
    
    return out_image
            
res = RGB_median(lena_rgb_szum, 3)

In [ ]:
def plot_img(img, title=''):
    plt.figure(figsize=(img.shape[0]/30,img.shape[1]/30))
    plt.imshow(img, vmin=0, vmax=255)
    plt.axis('off')
    plt.title(title)
    plt.show()
    
plot_img(lena_rgb_szum)
plot_img(res)

In [ ]:
res1 = RGB_median(lena_rgb, 9)
plot_img(lena_rgb)
plot_img(res1)